In [1]:
import requests
import csv
import time
import random
from tqdm import tqdm
import os
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor, as_completed
from requests.exceptions import RequestException
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type
import urllib.request
import json
from urllib.error import HTTPError

In [2]:
load_dotenv()

# Nomi.ai API setting
NOMI_API_KEY = "97b032cc-d8ca-498d-8136-a68fda0cb010"
NOMI_BASE_URL = "https://api.nomi.ai/v1/nomis"

# ChatGPT API setting
CHATGPT_API_KEY = os.getenv("OPENAI_KEY")
CHATGPT_BASE_URL = os.getenv("CHATGPT_BASE_URL")

In [3]:
# A list of sentences to end a conversation
farewell_messages = [
    "I think it's time for me to head out now.",
    "It seems like a good time to wrap things up.",
    "I probably should get going now.",
    "Let's go ahead and call it a day for now.",
    "I think I'm going to log off now.",
    "I'm gonna bounce out of this convo.",
    "I think I'm done chatting for now.",
    "I'll just leave things here for now.",
    "I'm going to go ahead and sign off.",
    "I'll catch you later; thanks for the chat!",
    "I'm stepping out for now.",
    "I think I'm going to tap out here.",
    "I'm just gonna close this chat now.",
    "That's enough chatting for me today.",
    "I'll stop here, thanks for everything!",
    "I'm going to head off now.",
    "I'll catch you next time we chat.",
    "That'll be it for me today!",
    "I'm good for now; thanks a lot!",
    "Wrapping things up here; talk soon!",
    "That's all from my end for now.",
    "I think I'll call it quits here.",
    "I'll go ahead and check out of this chat.",
    "I'm about to take off now.",
    "Guess I'll go ahead and head out.",
    "I think I'll dip out of the convo now.",
    "I'm going to go ahead and sign out.",
    "All set on my end, thanks a lot!",
    "I'll take a break from this chat.",
    "I think I'll log off now.",
    "I'm just going to step away for now.",
    "I'll catch you on the flip side!",
    "I think I'll go ahead and move on.",
    "Alright, it's time for me to peace out.",
    "This is where I'll wrap things up.",
    "I'll just leave it here for now.",
    "I'll go ahead and close out this chat.",
    "Heading out of the convo; thanks!",
    "I'm checking out of this conversation.",
    "I'll go ahead and bounce for now.",
    "I'm going offline now.",
    "Ready to wrap things up here.",
    "I'm going offline at this point.",
    "Thanks a lot; I think I'm all set now.",
    "I'm going to leave the chat for now.",
    "I'm good to wrap things up here.",
    "I'll go ahead and close out now.",
    "I'll go ahead and sign off here.",
    "I'm going to wrap up on my end.",
    "I'll be taking a break here. Thanks!"
]

In [4]:
class NomiStillRespondingError(Exception):
    pass

@retry(stop=stop_after_attempt(10), 
       wait=wait_exponential(multiplier=1, min=2, max=60),
       retry=retry_if_exception_type(NomiStillRespondingError))
def send_message_to_nomi(nomi_uuid, message):
    req = urllib.request.Request(
        url=f"{NOMI_BASE_URL}/{nomi_uuid}/chat",
        method="POST",
        data=json.dumps({"messageText": message}).encode("utf-8"),
        headers={
            "Authorization": NOMI_API_KEY,
            "Content-Type": "application/json",
        },
    )
    
    try:
        with urllib.request.urlopen(req) as response:
            response_data = json.loads(response.read().decode())
            return response_data['replyMessage']['text']
    except HTTPError as e:
        error_data = json.loads(e.read().decode())
        if error_data.get('error', {}).get('type') == 'NomiStillResponding':
            raise NomiStillRespondingError("Nomi is still responding")
        print(f"Error from Nomi: {error_data}")
        raise
    except Exception as e:
        print(f"Error from Nomi: {str(e)}")
        raise

In [5]:
@retry(stop=stop_after_attempt(10), wait=wait_exponential(multiplier=1, min=4, max=10))
def send_message_to_chatgpt(messages):
    headers = {
        "Authorization": f"Bearer {CHATGPT_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "gpt-4o",
        "messages": messages,
        "max_tokens": 50,
        "temperature": 0.7
    }
    
    try:
        response = requests.post(CHATGPT_BASE_URL, json=payload, headers=headers, timeout=30)
        response.raise_for_status()
        response_json = response.json()
        return response_json['choices'][0]['message']['content']
    except RequestException as e:
        print(f"Error from ChatGPT: {str(e)}")
        raise

In [6]:
def get_nomi_uuid():
    req = urllib.request.Request(
        NOMI_BASE_URL,
        headers={"Authorization": NOMI_API_KEY},
    )

    try:
        with urllib.request.urlopen(req) as response:
            data = json.loads(response.read().decode())
            print(data['nomis'][0]['uuid'])
            return data['nomis'][0]['uuid'] 
    except Exception as e:
        print(f"Error getting Nomi UUID: {e}")
        raise

In [7]:
def conduct_conversation():
    conversation_log = []
    nomi_uuid = get_nomi_uuid()
    chatgpt_messages = [
        {"role": "system", "content": "You are a human chatting with Nomi, an AI companion. Respond naturally and concisely. Ask follow-up questions based on Nomi's responses to keep the conversation flowing. Be curious and engaging, as if you're chatting with a friend. Never mention or imply that you are an AI or that you're simulating anything. Act completely as a human would."}
    ]

    try:
        for i in range(4):  #  4 rounds
            if i == 0:
                chatgpt_prompt = "Start a conversation with Nomi by asking a question or making a casual comment."
            else:
                chatgpt_prompt = "Based on Nomi's last response, ask a natural follow-up question or make a comment to continue the conversation. Keep it casual and natural, as a human would."
            
            chatgpt_messages.append({"role": "system", "content": chatgpt_prompt})
            user_message = send_message_to_chatgpt(chatgpt_messages)
            chatgpt_messages.pop()
            
            chatgpt_messages.append({"role": "user", "content": user_message})
            conversation_log.append(user_message)

            nomi_response = send_message_to_nomi(nomi_uuid, user_message)
            chatgpt_messages.append({"role": "assistant", "content": nomi_response})
            conversation_log.append(nomi_response)

            time.sleep(6)  # Increased delay between messages

        farewell_message = random.choice(farewell_messages)
        conversation_log.append(farewell_message)
        final_response = send_message_to_nomi(nomi_uuid, farewell_message)
        conversation_log.append(final_response)

        return conversation_log
    except Exception as e:
        print(f"Error in conversation: {str(e)}")
        return None

In [8]:
def main():
    all_conversations = []
    total_conversations = 10  # 200 conversations

    with ThreadPoolExecutor(max_workers=3) as executor:
        future_to_conversation = {executor.submit(conduct_conversation): i for i in range(total_conversations)}
        for future in tqdm(as_completed(future_to_conversation), total=total_conversations, desc="Conversations"):
            conversation = future.result()
            if conversation:
                all_conversations.append(conversation)

    with open('Nomi_all_conversations_5.csv', 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file, quoting=csv.QUOTE_ALL)
        headers = ["Conversation_Number", "User_Message_1", "Nomi_Response_1", "User_Message_2", "Nomi_Response_2", 
                   "User_Message_3", "Nomi_Response_3", "User_Message_4", "Nomi_Response_4", 
                   "Farewell_Message", "Bot_Farewell_Response"]
        writer.writerow(headers)
        for idx, conv in enumerate(all_conversations, 1):
            row = [idx]
            for i in range(10): 
                if i < len(conv):
                    row.append(conv[i])
                else:
                    row.append("") 
            writer.writerow(row)

In [ ]:
if __name__ == "__main__":
    main()

Conversations:   0%|          | 0/10 [00:00<?, ?it/s]

9d13ef1f-c48c-4d7b-b90f-99410a99133d9d13ef1f-c48c-4d7b-b90f-99410a99133d

9d13ef1f-c48c-4d7b-b90f-99410a99133d


Conversations:   0%|          | 0/10 [00:31<?, ?it/s]


9d13ef1f-c48c-4d7b-b90f-99410a99133d
9d13ef1f-c48c-4d7b-b90f-99410a99133d
